In [ ]:
## Chapter04から使用するテキストデータをコピーする（事前にChapter04/Sentiment.ipynbを実行してください）
!cp -r ../../Chapter04/sisyou_db ./

In [ ]:
import spacy
import pandas as pd
import numpy as np


## モデルのロードとデータの読み込み
nlp = spacy.load("ja_ginza")
df = pd.read_excel("sisyou_db/sisyou_db_h29_01.xlsx", skiprows=[1])
df.head()

In [ ]:
## 文書ベクトルの表示
text = df["災害状況"][0]
print(text)

doc = nlp(text)
print(doc.vector)

In [ ]:
## 文書間類似度の算出
texts = [
    "工場内で、鉄骨階段（仮組）高さ2m50cmに手摺を仮組立作業中、足を滑らせて転倒し、顔面を強打して骨折し、右脇腹肋骨も骨折した。",
    "倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って転倒し、階段を転げ落ち（4段位）、持っていた荷物を足に落としてしまい、右足の腓骨を骨折した。",
    "道路の3車線の真ん中を走行中、左車線に侵入してしまい、走行中の大型ワンボックスカーと衝突し、首と左肩を痛め、回転性のめまいで入院し、痺れもある。"
]
docs = list(nlp.pipe(texts))

for i in range(len(docs)-1):
    for j in range(i+1, len(docs)):
        print(i, j, docs[i].similarity(docs[j]))

In [ ]:
## 文書間類似度行列の作成
from sklearn.preprocessing import normalize

vecs = [doc.vector for doc in nlp.pipe(df["災害状況"])]
X = np.array(vecs)
X = normalize(X, axis=1, norm='l2')

cos_sims = X @ X.T
print(cos_sims)

In [ ]:
## 類似度行列を用いて類似文書検索
target = 9
print(target, df["災害状況"][target])

top_index = np.argsort(cos_sims[target])[::-1]
print(df["災害状況"][top_index][1:10]) #top_index[0]は入力文書